# Строение CNN

Свёрточные нейронные сети обычно глубокие: в них не меньше десятка слоёв. Однако если рассмотреть разные архитектуры CNN, то все они состоят из двух частей: свёрточной и полносвязной. 

Свёрточная часть состоит из слоёв свёртки и пулинга и предназначена для поиска признаков на изображении. Пройдя через эти слои, изображение трансформируется в вектор признаков.

Вектор признаков — это найденные на изображении признаки, выраженные в виде чисел. К примеру, если в векторе 128 чисел, то:

- Первое может означать наличие или отсутствие на изображении кота. Чем больше число — тем кот более заметен.
   
- Второе — насколько красные объекты на фото. Чем число больше, тем больше красных объектов.
    …и так далее.

Причём только нейросеть знает, какое число за какой признак отвечает — для разработчика это непрозрачно.

Полносвязная часть состоит из полносвязных слоёв и принимает решение, основываясь на векторе признаков. Она работает по правилам обычного многослойного перцептрона (MLP): получает на вход числовые признаки, выдаёт результат решения задачи. Её архитектура выбирается по обычным правилам для MLP.

## Классификация

>**Классификация** — это отнесение изображения к одному или нескольким классам из заданного списка. Например:

- определение марки автомобиля по фото;
  
- распознавание рецепта по фото блюда;

- распознавание названия цветка по фото.

Целевая переменная для задачи классификации — это **метка класса.**

Датасеты для классификации состоят из пар «изображение — класс». MNIST — типичный пример датасета для классификации.

Класс обычно представляется в one-hot encoding (унитарное кодирование) — как вектор нулей с одной единицей на позиции, соответствующей номеру класса. Длина вектора — количество классов. Примеры:

- [0, 0, 1, 0] — означает класс 2 из четырёх возможных;
  
- [1, 0, 0] — это класс 0 из трёх возможных.

Это удобно, потому что можно интерпретировать числа как вероятность принадлежности изображения классу: 1 — 100% является этим классом, 0 — 0%, точно не является этим классом.

Соответственно, модель должна выдавать вектор такого же типа на выходном слое. Для этого в выходном слое должно быть столько нейронов, сколько классов в задаче. Например, если классов 10, то нужен слой nn.Linear(N, 10), где N — количество признаков в векторе, полученном из свёрточной части.

Чтобы значения результатов выходного слоя были в интервале [0, 1], к нему нужно добавить одну из типичных функций активации для задачи классификации: nn.Sigmoid() или n.Softmax(dim=1).

- **nn.Sigmoid()** используется для бинарной классификации или случаев, когда одному изображению может соответствовать несколько классов. Эта функция для каждого переданного в неё значения возвращает число в интервале [0, 1].

- **nn.Softmax(dim=1)** используется для остальных видов классификации. Эта функция сначала для каждого переданного значения рассчитывает nn.Sigmoid(), а затем нормализует вектор результата, то есть делит его на сумму его значений. В результате получается вектор типа [0.7, 0.2, 0.1] — каждое число лежит в интервале [0, 1] и при этом сумма всех чисел равна 1. Если округлить все числа, получится как раз вектор [1, 0, 0] — такого же вида, как целевой. Именно поэтому для функции указывается dim=1. Это обеспечивает нормализацию по изображениям: для каждого входного изображения получится вектор классов с суммой, равной 1. В противном случае была бы нормализация по классам: сумма значений класса по всем изображениям была бы 1, что бессмысленно.

Типичная архитектура CNN для классификации, если опустить свёрточную часть, выглядит так:

```python
import torch.nn as nn
model = nn.Sequential(
          # Здесь свёрточная часть, которая возвращает вектор с 512 признаками
          nn.Linear(512, 10),
          nn.Softmax(dim=1),
        )
```

## Детекция

Детекция, или детектирование, — это определение местоположения объекта на изображении. Например:

- подсчёт посетителей магазина по кадрам с видеонаблюдения;
   
- отслеживание питомца на кадрах с видеонаблюдения;
  
- распознавание координат машины в потоке, чтобы рассчитать её скорость и оштрафовать нарушителей.

Обычно результат решения задачи — координаты прямоугольника, в который вписан объект. Такой прямоугольник называется ограничивающим боксом или bounding box, bbox.

Соответственно, датасет для детекции состоит из пар «изображение — список боксов». Известный пример — датасет COCO.

В датасете боксы могут быть представлены в разных форматах:

- координаты левого верхнего и нижнего правого углов;

- координаты центра + ширина и высота;

- координаты верхнего левого угла + ширина и высота.

Однако это всегда 4 числа, достаточные, чтобы задать прямоугольник. Для передачи боксов в модель в качестве целевой переменной, они преобразуются в формат [cx , cy, w, h], где:

- cx — координата центра бокса по горизонтали (оси X),
  
- cy — координата центра бокса по вертикали (оси Y),
   
- w — ширина бокса,
    
- h — высота бокса.

Тогда для детектирования одного объекта на изображении полносвязная часть архитектуры может выглядеть так:

```py
import torch.nn as nn
model = nn.Sequential(
          # Здесь свёрточная часть, которая возвращает вектор с 512 признаками
          nn.Linear(512, 512),
          nn.ReLU(),
          nn.Linear(512, 4)
        ) 
```
В ней 512 признаков анализируются одним полносвязным слоем с активацией ReLU, после чего выходной слой возвращает 4 числа, соответствующих координатам бокса. Современные детекторы устроены сложнее и содержат дополнительные ухищрения, чтобы искать объекты быстрее и меньше ошибаться, однако суть — предсказание бокса — остаётся такой же.

**Обратите внимание, что функция активации не используется, так как получаемые координаты ничем не ограничены и могут быть любым числом. Иногда используется nn.ReLU() на выходе, чтобы детектор не генерировал невозможные отрицательные координаты.**

>Если нужно находить несколько объектов, то количество нейронов выходного слоя умножается на количество объектов. Например, если один объект — это 4 нейрона, то два — это 8 нейронов и так далее. Поэтому архитектура моделей для детектирования сложнее и содержит больше параметров, чем для классификации.

## Сегментация

Сегментация — это определение на изображении групп пикселей, соответствующих объектам. Поэтому результат сегментации — это маска, равная по площади входному изображению. Значения маски соответствуют классу соответствующего пикселя на изображении. К примеру: 1 — объект, 0 — фон.

Датасеты для сегментации хранятся в формате «изображение — маска». Так как это более сложная задача, чем детекция, где достаточно приблизительного местоположения объекта, часто по датасетам для сегментации рассчитывают боксы и используют их в детекции. 

Упомянутый ранее COCO — это изначально датасет для сегментации: в нём объекты размечены именно масками, то есть указаны точные границы для каждого объекта.

Однако COCO легко использовать для детекции — достаточно рассчитать прямоугольник, в который вписывается маска объекта. Поэтому его используют и для детекции, и для сегментации, и это частая практика применения датасетов для сегментации.

В отличие от детекции и классификации модель должна выдавать не числа, а одноканальное изображение — маску. Полносвязные нейросети плохо справляются с передачей и анализом пространственного положения признаков на фото. Поэтому в архитектурах для сегментации полносвязной части нет, вместо неё ещё одна свёрточная.

Идея такова: из вектора признаков, полученного после первой свёрточной части, можно получить изображение, если использовать операцию, обратную свёртке. То есть архитектура сети для сегментации состоит из свёрточной и «развёрточной» частей.

«Развёрточная часть» состоит из операций upsampling (противоположность пулинга) и транспонированной свёртки (transposed-convolution). Несмотря на название, такая свёртка просто обратна обычной свёртке. Вместо того чтобы умножить фрагмент изображения на фильтр, суммировать и получить число, нужно умножить число на фильтр и получить фрагмент изображения.

Такие архитектуры значительно сложнее и подходят для узкого круга задач, поэтому в рамках этого модуля будут встречаться только самые популярные архитектуры — предназначенные для классификации.